In [2]:
import numpy as np
from numpy.linalg import inv
from pathlib import Path
import pandas as pd
import pickle

In [12]:
# load in the data that is needed for the model
p = Path('.').resolve().parent / 'lasso_and_n4sid' / 'n4sid_v10' /'output' 
if p.exists():
    a_matrix = np.load(p / 'matrix_A1.npy')
    b_matrix = np.load(p / 'matrix_B1.npy')
    c_matrix = np.load(p / 'matrix_C1.npy')
    d_matrix = np.load(p / 'matrix_D1.npy')

# for v10 d needs to be reset
d_matrix = np.array([[0, 0, 0, 0, 0]])

print(f"A: {a_matrix.shape} {a_matrix}")
print(f"B: {b_matrix.shape} {b_matrix}")
print(f"C: {c_matrix.shape} {c_matrix}")
print(f"D: {d_matrix.shape} {d_matrix}")

# u1test = pd.read_csv(p / 'u1test.csv')
# display(pd.__version__)
# display(u1test.columns)
# # u1test[u1test.columns[3:]] = u1test[u1test.columns[3:]].apply(pd.to_numeric, errors='coerce')
# display(u1test[u1test.columns[3:]].describe())

# load in the saved off mpc run
results_file = Path('.').resolve() / 'results' / '001_som3_mpc_boptest.pkl'
if p.exists():
    with open(results_file, 'rb') as f:
        results = pickle.load(f)

variables = results['mpc'].result_queries
print(f"{'Type':>11}{'Name':>20}")
print(f"{'----':>11}{'----':>20}")
for v in variables['ind']:
    if len(v) > 2:
        continue
    print(f"{v[0]:>10}:{v[1]:>20}")
    
    
# display(results['mpc'].data_fields)
_u_heating_powers = results['mpc']['_u', 'heating_power'][0:10]
_x_xs = results['mpc']['_x', 'x'][0:10]
_x_t_indoors = results['mpc']['_x', 't_indoor'][0:10]
_tvp_t_oa = results['mpc']['_tvp', 'TDryBul'][0:10]
_tvp_hglohor = results['mpc']['_tvp', 'HGloHor'][0:10]
_tvp_occ_ratio = results['mpc']['_tvp', 'occupancy_ratio'][0:10]
# _tvp_fan_power = results['mpc']['_tvp', 'P1_FanPow'][0:10]
# _tvp_int_gains = results['mpc']['_tvp', 'P1_IntGaiTot'][0:10]
_tvp_oa_vent = results['mpc']['_tvp', 'OAVent'][0:10]

# display(_u_heating_powers)

A: (7, 7) [[ 1.0180985  -0.13712866  0.00964417 -0.02490702 -0.01244884 -0.00746675
   0.00594889]
 [ 0.21029735 -0.67540431  0.61318863  0.04603134  0.11338978  0.01970418
   0.01287145]
 [-0.03078653  0.2620393   0.61610645 -0.04256249  0.03083964  0.09690574
  -0.01219877]
 [-0.0381004   0.40455231 -0.13583864  0.77955861 -0.20151526  0.04665674
   0.10966995]
 [ 0.03397471 -0.23051572  0.17234046 -0.0844447   0.78283097 -0.15173907
   0.15837601]
 [ 0.2106847  -1.02544643 -0.4412389  -1.20080819 -0.76919889  0.24059444
   0.27888119]
 [-0.04947028  0.44720994 -0.36384021 -0.18082755 -0.24635447  0.13150181
   0.63996764]]
B: (7, 5) [[-1.54843244e-02 -9.66247272e-04  6.70355301e-02  1.23018752e-03
  -2.00691377e-03]
 [-4.65968023e-02 -1.66978137e-03  9.34614198e-01  4.72041499e-03
  -3.23392665e-03]
 [-1.31348668e-01 -5.26271610e-03 -1.97157589e-01  5.01254672e-03
  -9.65440957e-03]
 [ 1.16278970e-01  3.40612327e-04 -2.21872261e-01 -7.92026989e-03
   1.10978008e-02]
 [ 9.56789624e-0

In [14]:
def x_next(_x, _u):
    return a_matrix @ _x + b_matrix @ _u

def y_pred(_x, _u):
    return c_matrix @ _x + d_matrix @ _u

def covariance(sigma1): #, sigma2):
#     cov1_2 = sigma1 * sigma2
#     cov2_1 = sigma2 * sigma1
#     cov_matrix = np.array([[sigma1 ** 2, cov1_2],
#                            [cov2_1, sigma2 ** 2]])
    # with only one parameter, this is just a single entry matrix
    cov_matrix = np.array([[sigma1]])
    return np.diag(np.diag(cov_matrix))

x0 = _x_xs[0]
_x = x0
for step in range(1,10):
    # get the previous action
    u_action = _u_heating_powers[step-1]

    u_array = np.array([
        u_action,              # Heating Power (W), [0 - 6000]
        _tvp_t_oa[step].tolist()[0],       # T_OA (K)  - freezing outside
        _tvp_hglohor[step].tolist()[0],    # Horizontal Global Irradiance (W) 
        _tvp_occ_ratio[step].tolist()[0],  # No occupants [ 0 - 6]
        _tvp_oa_vent[step].tolist()[0],    # OA Volumetric flow rate (m3/s), [0.01 - 0.175]  # full outside air
#         _tvp_int_gains[step].tolist()[0],  # Internal gains convective flow (W), ?  [ 0 - 3000]
#         _tvp_fan_power[step].tolist()[0],  # Fan Power (W), ? [0 - 500]
    ])
    
    # calculate the new states
    x_prime = x_next(_x, u_array)
    
    # calculate the new y_modeled
    y_modeled = y_pred(_x, u_array)
    print(f"x_next: {x_next} -- x_actual: {_x_xs[step]}")
#     print(f"y_modeled: {y_modeled} -- y_actual: {_x_t_indoors[step]}")
#     print(x_prime - _x_xs[step])
#     print(y_modeled - _x_t_indoors[step])
    # update state matrix
    _x = x_prime


x_next: <function x_next at 0x108c830e0> -- x_actual: [-0.69364283 -0.06778593  0.04964521 -0.03426983 -0.00084396 -0.01714104
 -0.00459698]
x_next: <function x_next at 0x108c830e0> -- x_actual: [-0.70673953 -0.06943683  0.04861692 -0.03350966 -0.00323316 -0.02252491
 -0.01074863]
x_next: <function x_next at 0x108c830e0> -- x_actual: [-0.7155343  -0.07138966  0.04744008 -0.0343248  -0.00434399 -0.02182389
 -0.01246058]
x_next: <function x_next at 0x108c830e0> -- x_actual: [-0.72121063 -0.07229919  0.04659034 -0.03583754 -0.0047216  -0.01726122
 -0.01152106]
x_next: <function x_next at 0x108c830e0> -- x_actual: [-0.72473673 -0.07304649  0.04650577 -0.03712127 -0.00496023 -0.01200788
 -0.00868697]
x_next: <function x_next at 0x108c830e0> -- x_actual: [-0.72660576 -0.07300608  0.04690263 -0.03802284 -0.00489305 -0.00689997
 -0.00520068]
x_next: <function x_next at 0x108c830e0> -- x_actual: [-0.72767917 -0.07292979  0.04771559 -0.03826415 -0.00475696 -0.00362569
 -0.00176323]
x_next: <func

/Users/nlong/.pyenv/versions/3.7.9/envs/3.7.9-modrlc/lib/python3.7/site-packages/ipykernel_launcher.py:27: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


In [10]:
# z is our observations, that is y_measured
# For now pin this to y_measured
z = _x_t_indoors
display(z)  # ✓

error_est_t_indoor = 0.25
error_obs_t_indoor = 0.5  # Uncertainty in the measurement

# initial guesses
P = covariance(error_est_t_indoor)
display(P)
# there is an A*P thing... not sure i need it with one state variable???

# Initial State Matrix
X = _x_xs[0].tolist()
n = len(X)
print(f"X is {X}")
print(f"n is {n}")

for index, data in enumerate(range(1,2)):
    print(f"****** INDEX {index+1} ******")
    print(f"data: {data}")
    X = y_pred(X[0][0], X[1][0], t, a)
    # To simplify the problem, professor
    # set off-diagonal terms to 0.
    P = np.diag(np.diag(A.dot(P).dot(A.T)))
    print(f"P: {P}")
    

array([[293.        ],
       [293.47043064],
       [293.7880923 ],
       [293.64584693],
       [293.50500843],
       [293.38486128],
       [293.26598004],
       [293.21034773],
       [293.1553007 ],
       [293.11335764]])

array([[0.25]])

X is [-0.6798564719375251, -0.06561750665718871, 0.049151726786789834, -0.03675713964146294, 0.002866468171244342, -0.006106764481204285, 0.003235793775143783]
n is 7
****** INDEX 1 ******
data: 1


TypeError: 'float' object is not subscriptable

In [77]:

# This is from 
# https://medium.com/@jaems33/understanding-kalman-filters-with-python-2310e87b8f48

x_observations = np.array([4000, 4260, 4550, 4860, 5110])
v_observations = np.array([280, 282, 285, 286, 290])

z = np.c_[x_observations, v_observations]
display(z)

# Initial Conditions
a = 2  # Acceleration
v = 280
t = 1  # Difference in time

# Process / Estimation Errors
error_est_x = 20
error_est_v = 5

# Observation Errors
error_obs_x = 25  # Uncertainty in the measurement
error_obs_v = 6

def prediction2d(x, v, t, a):
    A = np.array([[1, t],
                  [0, 1]])
    X = np.array([[x],
                  [v]])
    B = np.array([[0.5 * t ** 2],
                  [t]])
    X_prime = A.dot(X) + B.dot(a)
    return X_prime


def covariance2d(sigma1, sigma2):
    cov1_2 = sigma1 * sigma2
    cov2_1 = sigma2 * sigma1
    cov_matrix = np.array([[sigma1 ** 2, cov1_2],
                           [cov2_1, sigma2 ** 2]])
    return np.diag(np.diag(cov_matrix))


# Initial Estimation Covariance Matrix
P = covariance2d(error_est_x, error_est_v)
A = np.array([[1, t],
              [0, 1]])

# Initial State Matrix
X = np.array([[z[0][0]],
              [v]])
n = len(z[0])
print(f"X is {X}")
print(f"n is {n}")

# start at t+1, t=0 is used for the prediction x_{n+1}
for index, data in enumerate(z[1:]):
    print(f"****** INDEX {index+1} ******")
    print(f"data: {data}")
    print(f"x[0][0]: {X[0][0]}")
    print(f"x[1][0]: {X[1][0]}")
    X = prediction2d(X[0][0], X[1][0], t, a)
    # To simplify the problem, professor
    # set off-diagonal terms to 0.
    P = np.diag(np.diag(A.dot(P).dot(A.T)))
    print(f"P: {P}")

    # Calculating the Kalman Gain
    H = np.identity(n)
    R = covariance2d(error_obs_x, error_obs_v)
    S = H.dot(P).dot(H.T) + R
    K = P.dot(H).dot(inv(S))

    # Reshape the new data into the measurement space.
    Y = H.dot(data).reshape(n, -1)

    # Update the State Matrix
    # Combination of the predicted state, measured values, covariance matrix and Kalman Gain
    X = X + K.dot(Y - H.dot(X))

    # Update Process Covariance Matrix
    P = (np.identity(len(K)) - K.dot(H)).dot(P)
    
#     print(K)
    print(f"state: {X}")

print("Kalman Filter State Matrix:\n", X)

array([[4000,  280],
       [4260,  282],
       [4550,  285],
       [4860,  286],
       [5110,  290]])

X is [[4000]
 [ 280]]
n is 2
****** INDEX 1 ******
data: [4260  282]
x[0][0]: 4000
x[1][0]: 280
P: [[425   0]
 [  0  25]]
state: [[4272.5]
 [ 282. ]]
****** INDEX 2 ******
data: [4550  285]
x[0][0]: 4272.5
x[1][0]: 282.0
P: [[267.73028884   0.        ]
 [  0.          14.75409836]]
state: [[4553.85054707]
 [ 284.29069767]]
****** INDEX 3 ******
data: [4860  286]
x[0][0]: 4553.850547072261
x[1][0]: 284.2906976744186
P: [[197.90294898   0.        ]
 [  0.          10.46511628]]
state: [[4844.15764332]
 [ 286.22522523]]
****** INDEX 4 ******
data: [5110  290]
x[0][0]: 4844.157643316824
x[1][0]: 286.22522522522524
P: [[158.41665089   0.        ]
 [  0.           8.10810811]]
state: [[5127.05898493]
 [ 288.55147059]]
Kalman Filter State Matrix:
 [[5127.05898493]
 [ 288.55147059]]


In [ ]:
# Plots from MPC results
# TODO -- deal with time
# print(results['mpc']['_time'])

# max_t = 15
# fig, ax = plt.subplots(figsize=(15, 2))
# plt.plot(results['mpc']['_x', 't_indoor'][:max_t], 'g--', label="Tin[t]")
# plt.plot(results['mpc']['_x', 't_indoor_1'][:max_t], 'b--', label="Tin[t-1]")
# plt.plot(results['mpc']['_x', 't_indoor_2'][:max_t], 'r--', label="Tin[t-2]")
# # plt.plot(results['mpc']['_tvp', 'TSetpoint_Upper'][:max_t], label="TUpper")
# # plt.plot(results['mpc']['_tvp', 'TSetpoint_Lower'][:max_t], label="Tlower")

# # plt.plot(index, row['EMA26'], marker='o', markersize=5, color="darkred")
# # plt.axvline(x=index, linewidth=2, color='r')

# plt.title('Indoor Temperatures')
# plt.xlabel('Time step (5 mins)')
# plt.ylabel('T (K)')
# plt.legend()

# test the y_meas calculation